Лексема - *острый*. 

Значения из словаря МАС:
1. (1) Имеющий хорошо колющий конец или хорошо режущий край; противоп. тупой.
2. (6) С большим количеством соли, пряностей, специй (о пище).
3. (7) Очень сильный, ясно, отчетливо ощущаемый. 

Диагностические слова: 
1. лезвие, нож, зуб, игла, точить
2. перец, блюдо, соус, горчица
3. боль, желание, радость, жалость

Скачала 3 датасета с диагн. словами из НКРЯ

In [0]:
!pip install git+https://github.com/lopuhin/python-adagram.git

In [0]:
!wget https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib -d adagram.joblib 

In [0]:
!pip install pymorphy2

In [0]:
!pip install pymystem3==0.1.10

In [0]:
!pip install nltk

In [87]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
import adagram
from allennlp.commands.elmo import ElmoEmbedder
import pandas as pd
from lxml import html
from nltk.tokenize import RegexpTokenizer
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from tqdm import tqdm
from sklearn.metrics import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.cluster import *
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian'))

def normalize_pm(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

In [0]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [0]:
meaning_1 = pd.read_csv("1.csv", sep=";", error_bad_lines=False)
meaning_2 = pd.read_csv("2.csv", sep=";", error_bad_lines=False)
meaning_3 = pd.read_csv("3.csv", sep=";", error_bad_lines=False)

In [32]:
meaning_1.head(5)

,Reversed left context,Reversed center,Left context,Center,Punct,Right context,Title,Author,Birthday,Header,Created,Sphere,Type,Topic,Publication,Publ_year,Medium,Ambiguity,Full context
0,"етинкторп ,вотог ьсуг отч ,ясьтидебу",мыртсо,"убедиться, что гусь готов, проткните",острым,,ножом бедро.,Рецепты национальных кухонь: Скандинавская кух...,NaN,NaN,Рецепты национальных кухонь: Скандинавская кухня,2000-2005,"бытовая, нехудожественная, производственно-тех...",рецепт,дом и домашнее хозяйство,NaN,NaN,электронный текст,омонимия снята,"Для того, чтобы убедиться, что гусь готов, про..."
1,"ьнечо ,оннелдем ьтазер одан ёе",мыртсо,"её надо резать медленно, очень",острым,,"лезвием, которое следует постоянно оправлять",Николай Щербатов-Коломин. Березовое чудо // «Н...,Николай Щербатов-Коломин,NaN,Березовое чудо,2004,"нехудожественная, учебно-научная",статья,искусство и культура,«Народное творчество»,2004,журнал,омонимия снята,Если в каком-то месте вам попадается тонкая бе...
2,йошьлоб и ― иворк в,йыртсо,в крови ― и большой,острый,,нож.,Запись LiveJournal (2004),NaN,NaN,Запись LiveJournal,2004,"нехудожественная, электронная коммуникация",блог,частная жизнь,NaN,NaN,электронный текст,омонимия снята,"В руках у безумца ― портфель, в котором лежат ..."
3,тюазерорп и укйенил тюавыдалкирп метаЗ,мыртсо,Затем прикладывают линейку и прорезают,острым,,ножом уложенные внахлест оба полотнища.,Елена Волкова. Тот самый линолеум // «Биржа пл...,Елена Волкова,NaN,Тот самый линолеум,2002,"нехудожественная, публицистика",статья,дом и домашнее хозяйство,«Биржа плюс свой дом» (Н. Новгород),2002.08.12,газета,омонимия снята,Затем прикладывают линейку и прорезают острым ...
4,и имазалг имынмузеб с акеволеч,мыртсо,человека с безумными глазами и,острым,,ножом в руках.,Григорий Горин. Иронические мемуары (1990-1998),Григорий Горин,1940,Иронические мемуары,1990-1998,художественная,мемуары,_,NaN,NaN,книга,омонимия снята,Только тут все замечают за спиной артиста хохо...


In [33]:
meaning_2.head(5)

,Reversed left context,Reversed center,Left context,Center,Punct,Right context,Title,Author,Birthday,Header,Created,Sphere,Type,Topic,Publication,Publ_year,Medium,Ambiguity,Full context
0,доп кешылырк хынирук яинелвотогирп ялд,мыртсо,для приготовления куриных крылышек под,острым,,соусом надо приготовить маринад.,«Даша» делится секретами -- побалуй близких вк...,NaN,NaN,«Даша» делится секретами -- побалуй близких вк...,2004,"нехудожественная, публицистика",рецепт,дом и домашнее хозяйство,«Даша»,2004,журнал,омонимия снята,Сначала для приготовления куриных крылышек под...
1,",цереп йикдалс ,ьрибми ,осям еонриж",йыртсо,"жирное мясо, имбирь, сладкий перец,",острый,,"перец, всяческие приправы.",Григорий Тарасевич. Фейжоада: кастрюля бразиль...,Григорий Тарасевич,NaN,Фейжоада: кастрюля бразильской истории,2014,"нехудожественная, публицистика",статья,дом и домашнее хозяйство,«Русский репортер»,2014,журнал,омонимия не снята,Туда отправляем мелко нарезанными репчатый лук...
2,",ьного ― ещбоов уреткарах оп",йыртсо,"по характеру вообще ― огонь,",острый,,перец!,Дина Рубина. Русская канарейка. Блудный сын (2...,Дина Рубина,1953,Русская канарейка. Блудный сын,2014,художественная,роман,_,NaN,NaN,книга,омонимия не снята,"А по характеру вообще ― огонь, острый перец! ..."
3,митэ к апутсод еморК,мыртсо,Кроме доступа к этим,острым,,"блюдам, протест не принесет ничего.",Виктор Пелевин. Бэтман Аполло (2013),Виктор Пелевин,1962,Бэтман Аполло,2013,художественная,роман,_,NaN,NaN,книга,омонимия не снята,"Кроме доступа к этим острым блюдам, протест не..."
4,"с ― енатсикаП в ,имарамьлак",мыртсо,"кальмарами, в Пакистане ― с",острым,,соусом карри.,"Александра Дараган. «Хлеб, помидор, сыр... и ч...",Александра Дараган,NaN,"«Хлеб, помидор, сыр... и что-нибудь ещё»",2012,"нехудожественная, публицистика",статья,"дом и домашнее хозяйство, история, производство",«Зеркало мира»,2012,журнал,омонимия не снята,"В Японии, что естественно для морской страны, ..."


In [34]:
meaning_3.head(5)

,Reversed left context,Reversed center,Left context,Center,Punct,Right context,Title,Author,Birthday,Header,Created,Sphere,Type,Topic,Publication,Publ_year,Medium,Ambiguity,Full context
0,лавилисереп модурт мишьлоб с ежу,еортсо,уже с большим трудом пересиливал,острое,,желание схватить Степашу за ворот,Андрей Волос. Недвижимость (2000) // «Новый Ми...,Андрей Волос,1955,Недвижимость,2000,художественная,роман,_,«Новый Мир»,2001,журнал,омонимия снята,"Я чувствовал неприятное сердцебиение, вызванно..."
1,и ― екчовед к юьтсогортс,мыртсо,строгостью к девочке ― и,острым,,желанием Павла Алексеевича посадить дочку,Людмила Улицкая. Казус Кукоцкого [Путешествие ...,Людмила Улицкая,1943,Казус Кукоцкого [Путешествие в седьмую сторону...,2000,художественная,роман,_,«Новый Мир»,2000,журнал,омонимия снята,Странная это была несуразица между аскетически...
2,"ьсалалед окьлот ,ьсалянем ен ьвобюл",йортсо,"любовь не менялась, только делалась",острой,,"болью в нём, болью, которая",митрополит Антоний (Блум). О христианстве (1995),митрополит Антоний (Блум),1914,О христианстве,1995,"нехудожественная, церковно-богословская",беседа,религия,NaN,NaN,книга,омонимия снята,"Когда мы были "" хорошими"", его любовь была лик..."
3,"как ,еж кат ( етсерк",яартсо,"кресте ( так же, как",острая,,боль в душе и в,митрополит Антоний (Блум). О христианстве (1995),митрополит Антоний (Блум),1914,О христианстве,1995,"нехудожественная, церковно-богословская",беседа,религия,NaN,NaN,книга,омонимия снята,"И эта смерть Христова на кресте ( так же, как ..."
4,,яартсО,,Острая,,боль ударила как ножом.,И. Грекова. Перелом (1987),И. Грекова,1907,Перелом,1987,художественная,повесть,_,NaN,NaN,книга,омонимия снята,Острая боль ударила как ножом. [И. Грекова. П...


In [53]:
len(meaning_2['Center']) # самый маленький датасет

72

Адаграм распределяет значения так: для первого - 3, для второго - 0, для третьего - 4.

In [0]:
all_meanings = []

for i in range(len(meaning_2['Center'])): ## везде одинаковое количество
    all_meanings.append((" ".join([meaning_1["Left context"][i].strip(), 
                                  meaning_1["Center"][i].strip(), 
                                  meaning_1["Right context"][i].strip()]), 3))
    
for i in range(len(meaning_2['Center'])):
    all_meanings.append((" ".join([meaning_2["Left context"][i].strip(), 
                                  meaning_2["Center"][i].strip(), 
                                  meaning_2["Right context"][i].strip()]), 0))
    
for i in range(len(meaning_2['Center'])):
    all_meanings.append((" ".join([str(meaning_3["Left context"][i]).strip(), 
                                  meaning_3["Center"][i].strip(), 
                                  meaning_3["Right context"][i].strip()]), 4))

In [56]:
len(all_meanings)

216

In [0]:
mystem = Mystem()

def disambiguate(model, word, context):
    word, _ = lemmatized_context(word)
    probs = model.disambiguate(word, lemmatized_context(context))
    return probs.argmax()


def lemmatized_context(s):
    return [w.lower() for w in mystem.lemmatize(" ".join(tokenize(s)))]

In [0]:
true_values = 0

for context in all_meanings:
    if context[1] == disambiguate(vm, 'острый', context[0]):
        true_values += 1

accuracy = true_values/len(all_meanings)

In [72]:
# неплохо
accuracy

0.8287037037037037

Теперь попробую для словарных толкований

In [0]:
from_dict_1 = ('Имеющий хорошо колющий конец или хорошо режущий край; противоп. тупой. Острая игла. Острая бритва. Острый нож. □ — Острые колючки рвали мою одежду. \
Лермонтов, Бэла. Они навязали их [крючки] на поводки из проволоки или струны, которых щуки не могли перекусить, несмотря на свои острые зубы. С. Аксаков, Детские годы Багрова-внука. \
Я растирал себе в кровь руки и колени о песок, об острые камни, о колючие сучки, но ничего не замечал. М. Пришвин, За волшебным колобком.', 3)
from_dict_2 = ('С большим количеством соли, пряностей, специй (о пище). Острые блюда. Острый сыр. || Едкий, пряный (о приправах). Острый перец. Острая горчица.', 0)
from_dict_3 = ('Очень сильный, ясно, отчетливо ощущаемый. Острое желание. Острая боль. □ Пашка взбежал на ступени, взглянул в окно, и острая, захватывающая радость вдруг овладела им. Чехов, Беглец. Острая жалость к себе наполнила ее сердце. С. Антонов, Дожди.', 4)

In [97]:
print('Угадал ли со значением в первом толковании?')
from_dict_1[1] == disambiguate(vm, 'острый', from_dict_1[0])

Угадал ли со значением в первом толковании?


True

In [98]:
print('Угадал ли со значением во втором толковании?')
from_dict_2[1] == disambiguate(vm, 'острый', from_dict_2[0])

Угадал ли со значением во втором толковании?


True

In [99]:
print('Угадал ли со значением в третьем толковании?')
from_dict_3[1] == disambiguate(vm, 'острый', from_dict_3[0])

Угадал ли со значением в третьем толковании?


True

Везде угадал